In [1]:
ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"

def MODULE_FROM_NOTEBOOK(target, source, *keys): 
    from json import load
    with open(source) as notebook:
        data = load(notebook)
        with open(target,'w') as module:
          for cell in data['cells']:
              lines = cell['source']
              if type(lines) == str:
                  lines = lines.split('\n')
              line0  = (lines or [''])[0].strip()
              haskey = lambda key: line0 == f'### {key} ###'
              if cell['cell_type'] == 'code' and any(map(haskey,keys)):
                  code = '\n'.join(lines)
                  module.write(code)
                  module.write('\n')
                     
MODULE_FROM_NOTEBOOK('./ttt_tactics.py',
                     './ttt-tactics.ipynb',
                     'HEAD','GAME','DATA','MODEL','SEARCH')

from ttt_tactics import *

ipynb = 'ttt-tactics-train'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks
__pycache__  ttt-tactics	ttt-tactics-play.ipynb	ttt-tactics-train.ipynb
ttt-3.ipynb  ttt-tactics.ipynb	ttt_tactics.py
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14921617294454268278
,
 name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1282729240238769022
physical_device_desc: "device: XLA_CPU device"
]
tensorflow: 2.3.0
DIR = ./ttt-tactics
MODULE =  ttt_tactics
<class 'ttt_tactics.AlphaZeroModel'>
<function AlphaZeroPolicy at 0x7fd40fb47d90>


In [ ]:
### TRAIN ###

model   = AlphaZeroModel()
minloss = 3.0
iters   = 10_000
while iters > 0:
    iters -= 1

    models = sortedfiles(f'{DIR}/model-*.h5')
    model.load(models[-1])
    num = int(re.findall(r'\d+', str(model))[0]) 
    print('num =', num)

    path    = f'{DIR}/data-*.tfrec'
    files   = sortedfiles(path)
    dataset = loadsamples(files, buffer_size=2048, batch_size=64, seed=None)
    print(f'found {len(files)} files at {path}')

    schedule = { 0:0.0001 }
    def scheduler(epoch, lr): 
        if epoch in schedule: return schedule[epoch]  
        else:                 return lr

    history = model.fit(
        x                   = dataset,
        epochs              = 10,
        verbose             = 1,
        callbacks           = [LearningRateScheduler(scheduler)],
        workers             = 10,
        use_multiprocessing = False)
    history = history.history

    loss = history['loss'][-1]
    ok   = loss < minloss
    if ok:
        minloss = loss
        model.save(f'{DIR}/model-{(num+1):0>3d}.h5')

    plt.figure()
    plt.plot(history['loss'],        label='total loss')
    plt.plot(history['policy_loss'], label='policy loss')
    plt.plot(history['value_loss'],  label='value loss')
    plt.title('training history')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc="upper left")
    plt.savefig(f'{DIR}/{str(model)}.png')
    plt.show()

    ok = 'OK' if ok else 'NO'
    NOTIFY(f'{str(model)} {ok}, minloss: {minloss}, loss: {loss}, shell={str(ipython)}', title=ipynb)


./ttt-tactics/model-005.h5 Oct 08 2020 13:07:39
num = 5
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Do not call `graph_parents`.
found 17 files at ./ttt-tactics/data-*.tfrec
Epoch 1/10
   2544/Unknown - 173s 68ms/step - loss: 2.3003 - value_loss: 0.7289 - policy_loss: 1.5714 - value_lr: 1.0000e-04